# CoexistAI Tool Tutorial

Welcome to the tutorial for the coexistAI tool! This notebook will guide you through the main functionalities of the tool, including web search, document processing, generative models, answer generation, YouTube summarization, and more. Each section contains explanations and code examples to help you get started quickly.

## 1. Setup and Initialization

First, let's import the required libraries, set up environment variables, and initialize the main components. This ensures that all dependencies are loaded and ready for use.

In [1]:
from utils.utils import *
set_logging(True) 
from langchain_text_splitters import TokenTextSplitter
import os

if not is_searxng_running():
       !docker run --rm \
                 -d -p 30:8080 \
                 -v "${PWD}/searxng:/etc/searxng" \
                 -e "BASE_URL=http://localhost:$PORT/" \
                 -e "INSTANCE_NAME=my-instance" \
                 searxng/searxng
else:
       print("SearxNG docker container is already running.")

os.environ['GOOGLE_API_KEY'] = 'YOUR_API_KEY'  # Replace with your actual key
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=128)
from utils.websearch_utils import *
searcher = SearchWeb(30)  # Initialize web search with a result limit

SearxNG docker container is already running.

[2025-06-07 19:35:15,485] WARNING langchain_community.utils.user_agent: USER_AGENT environment variable not set, consider setting it to identify your requests.


## 2. Loading Models

Load embedding models and cross-encoders using the `load_model` function. You can choose between different embedding modes such as 'gemini', 'huggingface', or 'infinity_emb'.

In [2]:
hf_embeddings, cross_encoder = load_model("models/embedding-001", _embed_mode='gemini')

[2025-06-07 19:35:15,718] INFO utils.utils: Loading model: models/embedding-001 with embedding mode: gemini
[2025-06-07 19:35:15,718] INFO utils.utils: Loading model: models/embedding-001 with embedding mode: gemini
/Users/sidhantthole/Documents/llama_index_exp/coexist/coexist_trial/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-06-07 19:35:20,841] INFO sentence_transformers.cross_encoder.CrossEncoder: Use pytorch device: mps


## 4. Web Search Integration

Use the `SearchWeb` class to perform web searches and retrieve results. This is useful for augmenting LLMs with up-to-date information from the web.

In [3]:
results = searcher.query_search("latest news in AI", num_results=3)
print(results)

[2025-06-07 19:35:24,031] INFO utils.websearch_utils: Search results for query 'latest news in AI': [{'snippet': 'Anthropic launches Claude AI models for US national security · Reddit sues Anthropic over AI data scraping · Tackling hallucinations: MIT spinout teaches AI to ...', 'title': 'AI News | Latest AI News, Analysis & Events', 'link': 'https://www.artificialintelligence-news.com/', 'engines': ['google'], 'category': 'general'}, {'snippet': 'News coverage on artificial intelligence and machine learning tech, the companies building them, and the ethical issues AI raises today.', 'title': 'AI News & Artificial Intelligence', 'link': 'https://techcrunch.com/category/artificial-intelligence/', 'engines': ['google'], 'category': 'general'}, {'snippet': "Google CEO Sundar Pichai downplays AI jobs threats, says 'it allows us to do more' · FILE — Jensen Huang, CEO of NVIDIA, speaks at NVIDIA GTC in San Jose.", 'title': 'Artificial Intelligence: Read latest news updates on AI ...', 'link'

[
    {
        'snippet': 'Anthropic launches Claude AI models for US national security · Reddit sues Anthropic over AI 
data scraping · Tackling hallucinations: MIT spinout teaches AI to ...',
        'title': 'AI News | Latest AI News, Analysis & Events',
        'link': 'https://www.artificialintelligence-news.com/',
        'engines': ['google'],
        'category': 'general'
    },
    {
        'snippet': 'News coverage on artificial intelligence and machine learning tech, the companies building 
them, and the ethical issues AI raises today.',
        'title': 'AI News & Artificial Intelligence',
        'link': 'https://techcrunch.com/category/artificial-intelligence/',
        'engines': ['google'],
        'category': 'general'
    },
    {
        'snippet': "Google CEO Sundar Pichai downplays AI jobs threats, says 'it allows us to do more' · FILE — 
Jensen Huang, CEO of NVIDIA, speaks at NVIDIA GTC in San Jose.",
        'title': 'Artificial Intelligence: Read latest news updates on AI ...',
        'link': 'https://indianexpress.com/section/technology/artificial-intelligence/',
        'engines': ['google'],
        'category': 'general'
    }
]

## 5. Document Conversion from URLs

Convert URLs into document objects using the `urls_to_docs` function. This allows you to process and analyze web content as structured documents.

In [4]:
docs = await urls_to_docs([
    "https://en.wikipedia.org/wiki/India",
    "https://en.wikipedia.org/wiki/Bangalore"
])
print(f"Loaded {len(docs)} documents.")
print(docs[0])

[2025-06-07 19:35:24,048] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/India
[2025-06-07 19:35:24,052] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/Bangalore
[2025-06-07 19:35:24,353] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/Bangalore with type text/html; charset=UTF-8
/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py:136: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
[2025-06-07 19:35:24,366] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/India with type text/html; charset=UTF-8
[2025-06-07 19:35:26,169] INFO utils.websearch_utils: Processed markdown for: https://en.wikipedia.org/wiki/Bangalore
[2025-06-07 19:35:26,182] INFO utils.websearch_utils: Processed markdown 

Loaded 2 documents.

Document(
    metadata={'source': 'https://en.wikipedia.org/wiki/India'},
    page_content='\n\nIndia - WikipediaJump to contentCoordinates: 21Â°N 78Â°E\ufeff / \ufeff21Â°N 78Â°E\ufeff / 
21; 78\n![Featured \n![Extended-protected \n\nFrom Wikipedia, the free encyclopediaCountry in South Asia\nThis 
article is about the country. For other uses, see India (disambiguation).\n\n| Republic of India*BhÄ\x81rat 
Gaá¹\x87arÄ\x81jya* | |\n| --- | --- |\n| Horizontal tricolour flag bearing, from top to bottom, deep saffron, 
white, and green horizontal bands. In the centre of the white band is a navy-blue wheel with 24 spokes. Flag  State
emblem | |\n| **Motto:**Satyameva Jayate\xa0(Sanskrit)"Truth Alone Triumphs"[1] | |\n| **Anthem:**\xa0Jana Gana 
Mana\xa0 Art the Ruler of the Minds of All People"[4][2] | |\n| **National song: Vande Mataram\xa0(Sanskrit)[c]**"I
Bow to Thee, Mother"[b][1][2] | |\n| Image of a globe centred on India, with India highlighted.\xa0\xa0Territory 
controlled by India\xa0\xa0Territory claimed but not controlled | |\n| Capital | New Delhi28Â°36â\x80²50â\x80³N 
77Â°12â\x80²30â\x80³E\ufeff / \ufeff28.61389Â°N 77.20833Â°E\ufeff / 28.61389; 77.20833 |\n| Largest city by city 
proper population | Mumbai |\n| Largest city by metropolitan area population | Delhi |\n| Official\xa0languages |  
|\n| Recognised regional\xa0languages | State level and Eighth Schedule[9] |\n| Native languages | 424 languages[g]
|\n| Religion  (2011)[11] |  |\n| Demonym(s) |  |\n| Government | Federal parliamentary republic |\n|  | |\n| 
â\x80¢\xa0President | Droupadi Murmu |\n| â\x80¢\xa0Prime Minister | Narendra Modi |\n|  | |\n| Legislature | 
Parliament |\n| â\x80¢\xa0Upper house | Rajya Sabha |\n| â\x80¢\xa0Lower house | Lok Sabha |\n| 
Independence\xa0from the United Kingdom | |\n|  | |\n| â\x80¢\xa0Dominion | 15 August 1947 |\n| â\x80¢\xa0Republic 
| 26 January 1950 |\n|  | |\n| Area | |\n| â\x80¢\xa0Total | 3,287,263\xa0km2 (1,269,219\xa0sq\xa0mi)[2][h] (7th) 
|\n| â\x80¢\xa0Water\xa0(%) | 9.6 |\n| Population | |\n| â\x80¢\xa02023 estimate | Neutral increase 
1,428,627,663[13] (1st) |\n| â\x80¢\xa02011\xa0census | Neutral increase  (2nd) |\n| â\x80¢\xa0Density | 429.8/km2 
(1,113.2/sq\xa0mi) (30th) |\n| GDP\xa0(PPP) | 2025\xa0estimate |\n| â\x80¢\xa0Total | Increase $17.647 trillion[16]
(3rd) |\n| â\x80¢\xa0Per capita | Increase $12,132[16] (119th) |\n| GDP\xa0(nominal) | 2025\xa0estimate |\n| 
â\x80¢\xa0Total | Increase $4.187 trillion[16] (4th) |\n| â\x80¢\xa0Per capita | Increase $2,878[16] (136th) |\n| 
Gini\xa0(2021) | Positive decrease\xa032.8[17]medium inequality |\n| HDI\xa0(2023) | 
Increase\xa00.685[18]medium\xa0(130th) |\n| Currency | Indian rupee (â\x82¹) (INR) |\n| Time zone | UTC+05:30 (IST)
|\n| Date format |  |\n| Calling code | +91 |\n| ISO 3166 code | IN |\n| Internet TLD | .in (others) 
|\n\n**India**, officially the **Republic of India**,[j][20] is a country in South Asia. It is the seventh-largest 
country by area; the most populous country since 2023;[21] and, since its independence in 1947, the world\'s most 
populous  Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the 
southeast, it shares land borders with Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and 
Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and 
Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.\n\nModern humans arrived on the 
Indian subcontinent from Africa no later than 55,000 years ago.[26][27][28] Their long occupation, predominantly in
isolation as hunter-gatherers, has made the region highly diverse.[29] Settled life emerged on the subcontinent in 
the western margins of the Indus river basin 9,000 years ago, evolving gradually into the Indus Valley Civilisation
of the third millennium BCE.[30] By 1200\xa0BCE, an archaic form of Sanskrit, an Indo-European language, had 
diffused

## 6. Using Generative Models

Initialize and use generative models like `ChatGoogleGenerativeAI` and `ChatOpenAI` for text generation tasks. The `get_generative_model` function helps you select and configure the model.

In [5]:
llmgoogle = get_generative_model(model_name='gemini-1.5-flash',
                    type='google',
                    _tools=None,
                    kwargs={'temperature': 0.1, 'max_tokens': None, 'timeout': None, 'max_retries': 2, 
                            'api_key': os.environ['GOOGLE_API_KEY'],
                            'generation_config':{"response_mime_type": "application/json"}})

# For local (ollama, for others I will add the support in coming weeks)

# llmlocal = get_generative_model(model_name='qwen:0.5b-chat',
#                     type='local',
#                     _tools=None,
#                     kwargs={'temperature': 0.1, 'max_tokens': None, 'timeout': None, 'max_retries': 2})

[2025-06-07 19:35:26,545] WARNING langchain_google_genai.chat_models: Unexpected argument 'generation_config' provided to ChatGoogleGenerativeAI.
[2025-06-07 19:35:26,545] WARNING langchain_google_genai.chat_models: Unexpected argument 'generation_config' provided to ChatGoogleGenerativeAI.
/var/folders/sh/v0nx5vbx4mv07k1h4mxmwxcw0000gn/T/ipykernel_43895/4078719958.py:1: UserWarning: WARNING! generation_config is not default parameter.
                generation_config was transferred to model_kwargs.
                Please confirm that generation_config is what you intended.
  llmgoogle = get_generative_model(model_name='gemini-1.5-flash',


In [6]:
web_response = await query_web_response(
    "Top news of today",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=[],
    local_mode=False,
    split=False
)
print(web_response[0])

[2025-06-07 19:35:28,091] WARNING utils.answer_generation: Structured output failed: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]. Falling back to prompt-based extraction.
[2025-06-07 19:35:28,398] ERROR utils.answer_generation: Both structured and prompt-based extraction failed: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
[2025-06-07 19:35:28,398] ERROR utils.websearch_utils: Error generating search response for query 'Top news of today': not enough values to unpack (expected 3, got 0)


None

## 9. YouTube Transcript Summarization

Summarize YouTube video transcripts using the `youtube_transcript_response` function. This is useful for extracting insights from long videos.

In [6]:
summary = youtube_transcript_response("https://www.youtube.com/watch?v=o8NiE3XMPrM&t=6648s", 
                                      'summarize in bullets and themes', llmgoogle)
print(summary)

TypeError: string argument expected, got 'ExpatError'

TypeError: string argument expected, got 'ExpatError'

[2025-06-07 19:35:29,580] INFO IPKernelApp: Exception in execute request:
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~/Documents/llama_index_exp/coexist/coexist_trial/lib/python3.13/site-packages/IPython/core/async_helpers.py:128, in _pseudo_sync_runner(coro)
    120 """
    121 A runner that does not really allow async execution, and just advance the coroutine.
    122 
   (...)    125 Credit to Nathaniel Smith
    126 """
    127 try:
--> 128     coro.send(None)
    129 except StopIteration as exc:
    130     return exc.value

File ~/Documents/llama_index_exp/coexist/coexist_trial/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3386, in InteractiveShell.run_cell_async(self, raw_cell, store_history, silent, shell_futures, transformed_cell, preprocessing_exc_tuple, cell_id)
   3382 exec_count = self.execution_count
   3383 if result.error_in_exec:
   3384    

## 10. Generating Maps

Generate maps with routes and points of interest using the `generate_map` function. You can visualize directions and locations directly in your notebook.

In [ ]:
# Example: Generate a map with a route and POIs
from utils.map import *
s = generate_map("M G Road, Bangalore", "Indiranagar, Bangalore")
from IPython.display import display, HTML
with open("output/map_with_route_and_pois.html") as f:
    html_content = f.read()
# display(HTML(html_content))

2025-06-06 19:40:54,384 [INFO] Found 3 probable locations for 'M G Road, Bangalore'.
2025-06-06 19:40:54,385 [INFO] Auto-selected location: Mahatma Gandhi Road, Tasker Town, Shivajinagar, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560001, India
2025-06-06 19:40:55,716 [INFO] Found 3 probable locations for 'Indiranagar, Bangalore'.
2025-06-06 19:40:55,719 [INFO] Auto-selected location: Indiranagar, Basaveshwaranagar, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560079, India
2025-06-06 19:40:56,064 [INFO] Route found between start and end coordinates.
2025-06-06 19:40:56,852 [INFO] Found 78 POIs near (12.9747828, 77.6096698).
2025-06-06 19:40:57,368 [INFO] Found 12 POIs near (12.9962979, 77.5452778).
2025-06-06 19:40:57,468 [INFO] Map generated and saved as 'map_with_route_and_pois.html'.
2025-06-06 19:40:57,468 [INFO] Generated route directions.


## 11. Advanced Query Handling

Handle advanced queries, such as generating detailed reports or toy examples, using `query_web_response`. This enables complex, multi-step reasoning and content generation.

In [ ]:
detailed_report = await query_web_response(
    "Give me end to end working for text diffusion model.",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-06-06 19:41:00,998] INFO utils.websearch_utils: Search phrases for query 'Give me end to end working for text diffusion model.': ['text diffusion models', 'text diffusion working', 'text diffusion process', 'text diffusion steps']
2025-06-06 19:41:00,998 [INFO] Search phrases for query 'Give me end to end working for text diffusion model.': ['text diffusion models', 'text diffusion working', 'text diffusion process', 'text diffusion steps']
[2025-06-06 19:41:01,794] INFO utils.websearch_utils: Search results for query 'text diffusion models': [{'snippet': 'by Q Yi · 2024 · Cited by 13 — Diffusion models are a kind of math-based model that were first applied to image generation. Recently, they have drawn wide interest in ...', 'title': 'Diffusion models in text generation: a survey', 'link': 'https://peerj.com/articles/cs-1905/', 'engines': ['google'], 'category': 'general'}, {'snippet': 'Gemini Diffusion is our state-of-the-art research model exploring what diffusion means for la

#### Answer: 
Text diffusion models are generative models that learn to generate text by refining noise step-by-step. The core 
idea involves corrupting a clean piece of text by gradually adding noise over several steps and then training a 
model to reverse this noise-adding process. This allows the model to generate realistic text by undoing the noise. 
Here's a detailed breakdown of the end-to-end working of text diffusion models: 

 1. **Forward Diffusion Process (Noise Addition):**
 *  The input text is incrementally corrupted by adding noise at each step until it becomes pure noise.
 *  Instead of Gaussian noise (used for images), token corruption is applied to text data. This involves randomly 
replacing words with noise tokens or changing parts of the sentence structure.

 2. **Reverse Diffusion Process (Denoising):**
 *  The model learns to reverse the noise-adding process, gradually restoring the text to its original form.
 *  The model iteratively refines the text, correcting errors that may occur anywhere in the sentence.
 *  The model predicts the entire noise to be removed in a given time step, rather than predicting the noise to be 
removed from the immediate previous step.

 3. **Mathematical Formulation:**
 *  The forward diffusion process can be defined as: q(xt|xt-1) = N(xt; sqrt(1-βt)xt-1, βtI), where xt is the 
output at step t, N is a normal distribution, βt is the noise schedule, and I is the identity matrix.
 *  Using reparameterization, noise can be added at any arbitrary step t without going through the iterative 
process.

 4. **Model Architecture:**
 *  A common architecture is a modified U-Net, which includes downsampling and upsampling blocks, ResNet blocks, 
and self-attention blocks.
 *  The model incorporates embeddings with information about the current time step and prompt (if the model 
supports prompting).
 *  A text encoder, such as CLIP, transforms input text prompts into embeddings, which guide the image generation 
process.

 5. **Training:**
 *  The model is trained to predict the noise added to the noisy image zt, given a set of inputs such as time step 
t and text prompt.
 *  The training objective is to minimize the mean squared error between the predicted noise and the actual noise 
added.

 6. **Inference:**
 *  The model starts with random noise and iteratively denoises it based on the learned reverse diffusion process.
 *  A scheduler controls the noise application and removal process during image generation.

 7. **Advantages:**
 *  Flexibility: DLMs offer new possibilities in terms of flexibility.
 *  Parallelism: DLMs offer exciting possibilities for parallel generation.
 *  Error correction: Step-by-step refinement allows the model to correct errors.

 8. **Components of Latent Diffusion (e.g., Stable Diffusion):**
 *   **VAE (Variational Autoencoder):** Used to encode images into a lower-dimensional latent space, reducing 
memory and computational requirements.
 *   **U-Net:** Takes noisy latent representations of images as input and predicts the noise present in them.
 *   **Text-encoder:** Transforms input text prompts into embeddings, guiding the image generation process.
 *  **Scheduler:**  Controls the noise application and removal process.

 9. **Comparison to Autoregressive Models:**
 *  Diffusion models generate the entire output at each step, whereas autoregressive models generate 
token-by-token.
 *  Diffusion models can be faster than autoregressive models at generating a large number of tokens.

 10. **Limitations:**
 *  Diffusion models are slower at ingesting long context windows.
 *  Challenges remain in applying diffusion models to text generation due to the discrete nature of text data.

 11. **Applications:**
 *  Text generation, correction, and inpainting.
 *  Image generation from text prompts (e.g., Stable Diffusion).
 *  Audio generation.

 12. **Key Concepts:**
 *   **Noise Schedule:**  Controls the amount of noise added at each step of the forward dif

In [ ]:
# You can even summarise full page 
detailed_report = await query_web_response(
    "Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0][0])

[2025-06-06 19:41:27,972] INFO utils.websearch_utils: Search phrases for query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['ollama.com secureminions summary', 'ollama.com secureminions learning']
2025-06-06 19:41:27,972 [INFO] Search phrases for query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['ollama.com secureminions summary', 'ollama.com secureminions learning']
[2025-06-06 19:41:27,973] INFO utils.websearch_utils: Extracted URLs from query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['https://ollama.com/blog/secureminions']
2025-06-06 19:41:27,973 [INFO] Extracted URLs from query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['https://ollama.com/blog/secureminions']
[2025-06-06 19:41:27,973] INFO utils.websearch_utils: Created Document for sourc

## Summary of "Secure Minions: Private Collaboration between Ollama and Frontier Models"

This blog post discusses Secure Minions, a project building upon the original Minions concept to enable private 
collaboration between local Ollama models and frontier models in the cloud.

**Original Minions:**

*   Connects local LLMs (like Gemma) to cloud-based frontier models (like GPT-4o).
*   Keeps raw context local, accessible only by the local LLM.
*   The frontier model orchestrates local LLMs and aggregates their outputs.
*   Reduces cloud costs by sending fewer tokens to the cloud (5x-30x savings).
*   Achieves 98% of frontier model accuracy.
*   Offers privacy benefits by keeping sensitive context on the device.

**Secure Minions:**

*   Encrypts the entire local-remote communication protocol end-to-end, protecting data even from the cloud 
provider.
*   Leverages the "confidential computing mode" of NVIDIA's Hopper H100 GPUs.
*   Ensures no plaintext is exposed during transmission or remote LLM inference.
*   Introduces minimal latency overhead (less than 1%, even with long prompts and large models).

**In essence, Secure Minions makes confidential LLM collaboration a reality.**

The post also provides instructions on how to get started with Secure Minions:

1.  Clone the repository.
2.  Create a virtual environment (optional).
3.  Install the Python package and dependencies.
4.  Install Ollama and the Gemma 3 model.
5.  Run the Streamlit app for an interactive demo.
6.  Use the `minions` package in Python for programmatic access.

## Learning Notes

*   **Local-First LLMs:** The post highlights the privacy advantages of local LLMs like Ollama, where sensitive 
data remains on the user's device.
*   **Federated LLMs:** Minions and Secure Minions demonstrate a federated approach to LLMs, combining the benefits
of local processing with the power of frontier models.
*   **Confidential Computing:** The use of NVIDIA's Hopper H100 GPUs' confidential computing mode is a key enabler 
for Secure Minions, ensuring data privacy even during remote inference.
*   **End-to-End Encryption:** Secure Minions emphasizes the importance of end-to-end encryption for protecting 
sensitive data in LLM applications.
*   **Cost Optimization:** The original Minions project demonstrates how to reduce cloud costs by strategically 
using local LLMs for context processing.
*   **Practical Implementation:** The post provides clear instructions and code examples for setting up and using 
Secure Minions, making it easy for developers to experiment with the technology.
*   **Potential Applications:** Secure Minions opens up new possibilities for LLM applications in sensitive domains
like healthcare, finance, and legal, where data privacy is paramount.

In [ ]:
# or multiple pages
detailed_report = await query_web_response(
    "Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-06-06 19:41:40,304] INFO utils.websearch_utils: Search phrases for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['ollama.com secureminions summary', 'github.com llama.cpp summary']
2025-06-06 19:41:40,304 [INFO] Search phrases for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['ollama.com secureminions summary', 'github.com llama.cpp summary']
[2025-06-06 19:41:40,305] INFO utils.websearch_utils: Extracted URLs from query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['https://ollama.com/blog/secureminions', 'https://github.com/ggml-org/llama.cpp']
2025-06-06 19:41:40,305 [INFO] Extracted URLs from query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['https://ollama.com/blog/secureminions', 'https://github.com/ggml-org/llama.cpp']
[2025-06-06 19:41:40,305] INFO utils.websearch_utils

Summarising using 2 documents

[2025-06-06 19:41:48,020] INFO utils.websearch_utils: Response generated for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp'.
2025-06-06 19:41:48,020 [INFO] Response generated for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp'.


Here's a summary of the two documents:

**Secure Minions: private collaboration between Ollama and frontier models:**

This blog post discusses a new security protocol called "Secure Minions" developed by Stanford's Hazy Research lab.
It builds upon the original Minions project, which connects local Ollama models to frontier models in the cloud to 
reduce costs and maintain privacy. Secure Minions encrypts the entire local-remote communication, ensuring no 
plaintext is exposed, even to the cloud provider. This is achieved using the "confidential computing mode" of 
NVIDIA's Hopper H100 GPUs. The overhead of this encryption is minimal, adding less than 1% latency. The post 
provides instructions on how to get started with Secure Minions, including cloning the repository, installing 
dependencies, and running the provided Streamlit app or using the `minions` package programmatically in Python.

**GitHub - ggml-org/llama.cpp:**

This is a GitHub repository for `llama.cpp`, a project focused on enabling LLM (Large Language Model) inference in 
C/C++ with minimal setup and high performance on various hardware. It serves as a playground for developing new 
features for the ggml library. The repository provides instructions for installation, obtaining and quantizing 
models (specifically in GGUF format), and using command-line tools (`llama-cli`, `llama-server`, etc.) for 
experimentation and serving. It supports various backends like Metal, BLAS, CUDA, and others for different target 
devices. The project also includes documentation, examples, and tools for benchmarking and measuring model quality.
The repository is licensed under the MIT license.

In [ ]:
# you can even do search/summary within local files

# or multiple pages
detailed_report = await query_web_response(
    "Summarise this document",
    '06-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[['documents/1706.03762v7.pdf']],
    local_mode=True,
    split=False
)
print(detailed_report[0])

[2025-06-06 19:41:55,621] INFO utils.websearch_utils: Search phrases for query 'Summarise this document': ['document summary']
2025-06-06 19:41:55,621 [INFO] Search phrases for query 'Summarise this document': ['document summary']
[2025-06-06 19:41:55,622] WARNING utils.websearch_utils: No websearcher provided; using document_paths only.
2025-06-06 19:41:55,622 [WARNING] No websearcher provided; using document_paths only.
[2025-06-06 19:41:55,623] INFO utils.websearch_utils: Starting context_to_docs for 1 URL groups.
2025-06-06 19:41:55,623 [INFO] Starting context_to_docs for 1 URL groups.
/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py:136: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
[2025-06-06 19:41:58,705] INFO utils.websearch_utils: Processing local file: documents/1706.03762v7.pd

Summarising using 2 documents

[2025-06-06 19:42:07,195] INFO utils.websearch_utils: Response generated for query 'Summarise this document'.
2025-06-06 19:42:07,195 [INFO] Response generated for query 'Summarise this document'.


The document is a research paper introducing the Transformer, a novel neural network architecture based solely on 
attention mechanisms, dispensing with recurrence and convolutions. The Transformer model achieves state-of-the-art 
results on machine translation tasks, surpassing existing models in quality while being more parallelizable and 
requiring less training time. The paper also demonstrates the Transformer's generalization capabilities by applying
it to English constituency parsing, achieving competitive results. Additionally, Google grants permission to 
reproduce the tables and figures in the paper for journalistic or scholarly works, provided proper attribution is 
given.

The next cell demonstrates how to summarize Reddit posts and comments from the "OpenAI" subreddit using the `reddit_reader_response` function. It retrieves the top 2 "hot" posts and summarizes their content with the help of the `llmgoogle` generative model. This is useful for quickly extracting key insights from Reddit discussions.

In [ ]:
from utils.reddit_utils import *
summary = reddit_reader_response(
  subreddit="OpenAI",
  url_type="hot",
  n=2,
  k=2,
  custom_url=None,
  time_filter="today",
  search_query=None,
  sort_type="hot",
  model=llmgoogle
)
print(summary)

Fetching posts from /r/OpenAI

Fetching comments for post: OpenAI is currently retaining all the chat data indefinitely - even for plus/pro users

Fetching comments for post: Did you see OpenAI's statement regarding their response to The New York Times?

## Summary of Reddit Posts and Comments: Objective: None

This summary covers all details from the provided Reddit posts and comments, irrespective of popularity or 
frequency of discussion.

**1. OpenAI Data Retention and the New York Times (NYT) Lawsuit:**

*   **Post 1:** A Reddit post 
((https://www.reddit.com/r/OpenAI/comments/1l4jvk3/openai_is_currently_retaining_all_the_chat_data/)) highlights 
that OpenAI is retaining all chat data indefinitely, even for Plus/Pro users. The post links to OpenAI's official 
response to data demands from the NYT ((https://openai.com/index/response-to-nyt-data-demands/)).
    *   **Comment 1.1:** A comment questions the legality of this data retention in the EU.
    *   **Comment 1.2:** Another comment suggests the data retention is specifically for the NYT case's legal 
purposes. It speculates that using the data for business training or other reasons would be illegal and unlikely. 
The comment downplays the impact on regular users, focusing on the increased server costs for OpenAI.

*   **Post 2:** A separate post 
((https://www.reddit.com/r/OpenAI/comments/1l4god3/did_you_see_openais_statement_regarding_their/)) asks if users 
have seen OpenAI's statement regarding their response to the NYT lawsuit, indicating that a lawsuit concerning 
ChatGPT user privacy is underway. The post also links to the same OpenAI statement 
((https://openai.com/index/response-to-nyt-data-demands/)).
    *   **Comment 2.1:** A comment expresses surprise at the post being downvoted, as it is simply sharing an 
official OpenAI statement.
    *   **Comment 2.2:** A comment in Italian criticizes the NYT for seemingly clinging to outdated logic and 
resisting cultural evolution. It argues that OpenAI is respecting its privacy policies and would be criticized 
regardless of whether it did or didn't. The comment suggests OpenAI should delete all data as per its policy and 
let the NYT accuse itself of "destruction of evidence." It concludes that journalism must adapt to AI rather than 
fight it with lawsuits, as change is inevitable.

---

This concludes the tutorial for the **coexist**. You have learned how to perform web search, document processing, answer generation, YouTube summarization, and more. For further details, refer to the project README or explore the codebase for advanced usage patterns. 